In [1]:
import pandas as pd
import numpy as np

In [2]:
# Importação dos Dados
names=['Data', 'Hora', 'Duração', 'DDD_Origem', 'ID_Origem', 'DDD_Destino', 'ID_Destino', 'Antena', 'Vazio', 'ID_Tipo_Ligação', 'Abrangência_Ligação', 'Operadora_Antena_Origem', 'Operadora_Antena_Destino', 'Categoria_USER', 'X']
dataset = pd.read_csv('../Dados/cdr_Uberlândia.txt', sep=';', names=names)

In [3]:
# Remoção de instâncias duplicadas e colunas inúteis.
dataset.drop_duplicates(inplace=True)
dataset.drop(['Vazio', 'X', 'ID_Tipo_Ligação', 'Abrangência_Ligação', 'DDD_Destino', 'DDD_Origem', 'Operadora_Antena_Origem', 'Operadora_Antena_Destino', 'Categoria_USER'], 
             axis=1, inplace=True)
dataset.head(3)

,Data,Hora,Duração,ID_Origem,ID_Destino,Antena
0,2013-04-16,08:10:26,0.55,F28A7CC2643468F06C81510658872E07,4EB08BB3316C38C499820B95DF87A725,55061
1,2013-04-16,13:38:23,0.77,565580EB8FDD16750497A4565ECDBFF8,C6C73DC14A20986650521214ADD8537D,55021
2,2013-04-16,09:18:36,4.05,565580EB8FDD16750497A4565ECDBFF8,C6C73DC14A20986650521214ADD8537D,55021


In [4]:
# Selecionando as ligações com duração maiores que 10 segundos
dataset = dataset[dataset['Duração'] >= 0.16]
dataset.shape

(13792553, 6)

In [5]:
dataset = dataset.groupby(['ID_Origem', 'ID_Destino']).filter(lambda x: len(x) >= 5)

In [6]:
agrup_origem = dataset.groupby(by='ID_Origem').filter(lambda x: len(x) <= 500)['ID_Origem']
agrup_origem.value_counts()

ID_Origem
98CFB759D141A2C9A554E93D9C262B3A    500
A79A70BDF9FC444794B5AE88B93F23F8    500
F6730452DB5B2D6C2683DFDEEB850C96    499
A91057B79D052FE07C0690F20838DD0A    499
161FB4ADAEA6AD07D766D012632D2423    499
                                   ... 
659F2025A58FD196A6D9E0EBD299E46C      5
FC22CF9F1F3B1C160C3C17963156E021      5
DC31D62949DA791A474013C546EACF4B      5
E5EF4FFF1DF0DF9CFF69F6A3016F71E3      5
3C1ACC5E015475DE21FDF1B95E1B9361      5
Name: count, Length: 162263, dtype: int64

In [7]:
agrup_destino = dataset.groupby(by='ID_Destino').filter(lambda x: len(x) <= 500)['ID_Destino']
agrup_destino.value_counts()

ID_Destino
7056934EAF493D57292F19D72485FFC6    491
06BFBD16BC9FE948D8AEA8202687A4FA    488
673D347524730EEE06BF69FD5AE89AA1    478
9911566765F9CFC25B46156409560204    476
EFD0779920C36ADD5A323011B752EE68    470
                                   ... 
536557B4CB993719D5BBF92CC03DC469      5
8DF631D1DDE95031FB84078028E680C5      5
98DCEE79720DCA8936A420EAFA7083A9      5
ECA030E7D2640773FA0EAFF215FDB157      5
9C481C23251264799C3637AE478DE70C      5
Name: count, Length: 338810, dtype: int64

In [8]:
# Todos os IDs que fizeram ou receberam mais de 5 ligações
nos = np.unique(np.append(agrup_origem, agrup_destino))
len(nos)

387894

In [9]:
dataset = dataset.query("ID_Origem in @nos | ID_Destino in @nos")
dataset.shape

(8171158, 6)

In [10]:
dataset.to_csv('cdr_processed_Uberlândia.csv', index=False)